## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
An unsupervised learning based approach to recover sparse graphs. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3


In [2]:
import torch
torch.__version__

'1.10.1'

### Create sample data

In [3]:
from uGLAD import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = main.get_data(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 10, 10), Samples (1, 500, 10)


### Running the uGLAD model

Until Convergence:  

    1. Initialize learnable `GLAD` parameters
    2. Run the GLAD model
    3. Get the glasso-loss
    4. Backprop

Possible reasons if `uGLAD` does not converge: 

    1. Lower the learning rate   
    2. Please re-run. This will run the optimization with different initializations  
    3. Change the INIT_DIAG=0/1 in the `GLAD` model parameters

In [10]:
from uGLAD import main
model_uGLAD = main.uGLAD_GL()  # Initialize the model
model_uGLAD.fit(Xb[0], centered=False)  # fit to the data

epoch:0/250 loss:13.6103515625
epoch:25/250 loss:13.477407455444336
epoch:50/250 loss:13.436330795288086
epoch:75/250 loss:13.373610496520996
epoch:100/250 loss:13.286905288696289
epoch:125/250 loss:12.010708808898926
epoch:150/250 loss:8.891230583190918
epoch:175/250 loss:8.086219787597656
epoch:200/250 loss:7.791337013244629
epoch:225/250 loss:7.632465839385986


In [11]:
print(model_uGLAD.precision_,'\n', model_uGLAD.precision_.shape)

[[ 1.5078534   0.441982   -0.         -0.          0.          0.
  -0.          0.38320804 -0.         -0.        ]
 [ 0.4419813   1.2526696   0.4506904  -0.          0.          0.
  -0.         -0.          0.5989577  -0.        ]
 [-0.          0.4506904   1.1888602   0.62595505 -0.          0.
   0.63859093 -0.         -0.         -0.        ]
 [-0.         -0.          0.6259555   1.4846597   0.51360565  0.
  -0.         -0.         -0.          0.        ]
 [ 0.          0.         -0.          0.5136047   1.4903675   0.
  -0.         -0.         -0.         -0.        ]
 [ 0.          0.          0.          0.          0.          1.783095
  -0.         -0.         -0.          0.50723124]
 [-0.         -0.          0.63859046 -0.         -0.         -0.
   1.4855901   0.         -0.          0.51358515]
 [ 0.38320798 -0.         -0.         -0.         -0.         -0.
   0.          1.5606968   0.          0.49283662]
 [-0.          0.5989577  -0.         -0.         -0.     

In [12]:
print(true_theta[0])

[[1.80824026 0.71610987 0.         0.         0.         0.
  0.         0.6128021  0.         0.        ]
 [0.71610987 1.80824026 0.74131555 0.         0.         0.
  0.         0.         0.77774837 0.        ]
 [0.         0.74131555 1.80824026 0.85566806 0.         0.
  0.8574103  0.         0.         0.        ]
 [0.         0.         0.85566806 1.80824026 0.72206713 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.72206713 1.80824026 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.80824026
  0.         0.         0.         0.70075488]
 [0.         0.         0.8574103  0.         0.         0.
  1.80824026 0.         0.         0.71725388]
 [0.6128021  0.         0.         0.         0.         0.
  0.         1.80824026 0.         0.57698523]
 [0.         0.77774837 0.         0.         0.         0.
  0.         0.         1.80824026 0.        ]
 [0.         0.         0.   